# Working with multiple characters

We will see how to duplicate the character, update their colors and move them in the scene

In [1]:
import numpy as np
from ipywidgets import widgets, interact, interactive

import ipyanimlab as lab
viewer = lab.Viewer(shadow_quality=lab.ShadowQuality.HIGH, move_speed=5, width=1200, height=720)

## Load the USD Asset
the default skinned character is accessible directly from internal resources

In [2]:
character_a = viewer.import_usd_asset('AnimLabSimpleMale.usd')

## Duplicate the asset

So we do not duplicate the GPU memory for the meshes

In [3]:
character_b = character_a.duplicate()
character_c = character_a.duplicate()

In [4]:
character_a.material_names()

['/AnimLabSimpleMale/mtl/SuitMaterial',
 '/AnimLabSimpleMale/mtl/ShirtMaterial',
 '/AnimLabSimpleMale/mtl/LeatherMaterial',
 '/AnimLabSimpleMale/mtl/ShoeMaterial',
 '/AnimLabSimpleMale/mtl/SkinMaterial']

## Update materials

So we can easily find the different characters

In [5]:
character_b.material('/AnimLabSimpleMale/mtl/SuitMaterial').set_albedo(np.array([0.5,1.0,0.5], dtype=np.float32))
character_c.material('/AnimLabSimpleMale/mtl/SuitMaterial').set_albedo(np.array([0.5,0.5,1.0], dtype=np.float32))

## Update the character transforms

We will move the different characters in the scene, so even if we render the same animation on each, they will be placed next to each others

In [9]:
xform = np.eye(4, dtype=np.float32)
xform[:3, 3] = [-100, 0, 0]
character_b.set_xform(xform)

xform[:3, 3] = [100, 0, 0]
character_c.set_xform(xform)

## Load the animation

we use the bvh animation with a mapper so we can render it easily

In [7]:
animmap = lab.AnimMapper(character_a, root_motion=True, match_effectors=True)
anim = lab.import_bvh('push1_subject2.bvh', anim_mapper=animmap)

## Render

we render the 3 characters exactly with the same animation here

In [8]:
def render(frame):
    
    p = (anim.pos[frame,...])
    q = (anim.quats[frame,...]) 
    a = lab.utils.quat_to_mat(q, p)
    
    viewer.begin_shadow()
    viewer.draw(character_a, a)
    viewer.draw(character_b, a)
    viewer.draw(character_c, a)
    viewer.end_shadow()
    
    viewer.begin_display()
    viewer.draw_ground()
    viewer.draw(character_a, a)
    viewer.draw(character_b, a)
    viewer.draw(character_c, a)
    viewer.end_display()
    
    viewer.execute_commands()
    
interact(
    render, 
    frame=lab.Timeline(max=anim.quats.shape[0]-1)
)
viewer

interactive(children=(Timeline(value=0, children=(Play(value=0, description='play', interval=33, layout=Layout…

Viewer(camera_far=2800.0, camera_near=20.0, camera_pitch=-18.0, camera_pos=[-370, 280, 350], camera_yaw=-45.0,…